In [51]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import re # 정규표현식
import urllib.request
from tqdm import tqdm # 작업 진행률 표시
from tensorflow.keras.preprocessing.text import Tokenizer # 텍스트 토큰화
from tensorflow.keras.preprocessing.sequence import pad_sequences # 길이가 다른 여러 문장의 길이를 임의로 동일하게 맞춰줌
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from konlpy.tag import Okt
from collections import Counter
okt = Okt()

In [63]:
# 클래스 선언
class Tokenizer:

    # 기본 생성자 선언
    def __init__(self, path, path2):
        self.path = path
        self.path2 = path2

        
    # 멤버메서드 선언
    def import_csv(self):
        # 데이터프레임으로 불러오기
        data = pd.read_csv(self.path, encoding='cp949')
        # 한글과 공백 제외 모두 제거 : 띄어쓰기 유지, 구두점 제거
        data['상품명'] = data['상품명'].str.replace('[^ㄱ-ㅎㅏ-ㅣ가-힣 ]',"")
   
        # 불용어가 제거된 리스트 
        bul_word = []
        # 불용어 제거        
        stopwords = ['의','가','이','은','들','을','하는','로','부터','처럼','만','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
        # 불용어 리스트 -> set
        stop_words = set(stopwords)
        for word_tokens in data['상품명']:
            for word in okt.morphs(word_tokens):
                if not word in stop_words:
                    bul_word.append(word)
        print(bul_word[:50])            
        self.remove_dlist(bul_word) # 다른 멤버메소드에 변수를 엑세스할 때 사용
    
  
    def remove_dlist(self, bul_word):

        # 명사 추출
        n_list = []
        for word in bul_word:
            nouns = okt.nouns(word)
            n_list.append(nouns)
            
        # 2중 리스트를 faltten 하게 만들기
        final_word = sum(n_list,[])
            
        # 빈도수 확인
        count = Counter(final_word)
        count_list = []
        
        for n, c in count.most_common(100):
            dics = {'word':n, 'count':c}
            count_list.append(dics)
        
        print(count_list[:5])

        self.make_df(count_list)

        
    # 최종 데이터프레임 만들기    
    def make_df(self, count_list):
        # 리스트 안에 딕셔너리의 원하는 키값만 사용하여 리스트를 만든 후 데이터프레임으로 만들기
        list1 = list(w['word'] for w in count_list)
        list2 = list(c['count'] for c in count_list)
        df_korean = pd.DataFrame(zip(list1, list2), columns=['word', 'count'])  
        print(df_korean[:5])
        self.export_csv(df_korean)
        
            
    # csv 파일로 내보내기
    def export_csv(self, df_korean):        
        df_korean.to_csv(self.path2, index = False, encoding='cp949')
        


In [64]:
if __name__=='__main__':
    path = 'D:\\VS_work\\MiniProject\\팀플_베스트셀러_자연어처리\\export_csv_file\\JG.csv'
    path2 = 'D:\\VS_work\\MiniProject\\팀플_베스트셀러_자연어처리\\export_csv_file\\J_count.csv'
    token = Tokenizer(path, path2)
    token.import_csv()

d:\anaconda\envs\second_evm\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  from ipykernel import kernelapp as app


['당신', '꿈', '무엇', '입', '니까', '멈추지', '마', '다시', '꿈', '써', '봐', '나', '결심', '하', '지만', '뇌', '비웃는다', '하루', '분', '정리', '힘', '서른', '살', '엔', '미처', '몰랐던', '것', '내', '행복해지는', '거절', '힘', '사람', '무엇', '성장하는가', '시크릿', '미생', '리딩', '리드', '하', '라', '어떻게', '원하는', '것', '얻는가', '미생', '나', '남', '무엇', '다른', '영어']
[{'word': '나', 'count': 560}, {'word': '법', 'count': 435}, {'word': '인생', 'count': 434}, {'word': '사람', 'count': 375}, {'word': '말', 'count': 369}]
  word  count
0    나    560
1    법    435
2   인생    434
3   사람    375
4    말    369
